<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match conventions expected by TracX.

In [1]:
import os
import sys

import numpy as np
import tensorflow as tf
from tifffile import imwrite

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.trk_io import load_trks

sys.path.append('..')
import utils

In [2]:
source_data = '../../data/test.trks'
data_dir = 'data'
ctc_dc_dir = os.path.join(data_dir, 'CTC', 'seg-dc')
ctc_gt_dir = os.path.join(data_dir, 'CTC', 'seg-gt')
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(data_dir, 'seg-gt')
seg_dc_dir = os.path.join(data_dir, 'seg-dc')

for d in [raw_dir, seg_gt_dir, seg_dc_dir, ctc_dc_dir, ctc_gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)
        
model_url = 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearSegmentation-7.tar.gz'

Load the test split of the tracking data

In [3]:
data = load_trks(source_data)

Load the DeepCell nuclear segmentation model to test the algorithm on predicted instead of ground truth segmentations

In [4]:
archive_path = tf.keras.utils.get_file('NuclearSegmentation.tgz', model_url, extract=True, cache_subdir='models')
model_path = os.path.splitext(archive_path)[0]
model = tf.keras.models.load_model(model_path)

app = NuclearSegmentation(model)

2022-12-01 00:22:45.849618: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 00:22:46.766899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1


In [5]:
# ChannelName_positionXXYYZZZ_timeTTTT.tif
name_template = '{}_position{:02}{:02}000_time{:05}.tif'

In [6]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))
    seg_dc_subdir = os.path.join(seg_dc_dir, '{:03}'.format(batch_no + 1))
    
    # Create directories if needed
    for d in (raw_subdir, seg_gt_subdir, seg_dc_subdir):
        if not os.path.exists(d):
            os.makedirs(d)
    
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)
    
    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]
    
    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]
    
    # Generate deepcell predictions
    y_pred = app.predict(X)
    
    # Save GT in CTC format
    for d in [ctc_dc_dir, ctc_gt_dir]:
        utils.save_ctc_gt(d, batch_no + 1, y[good_frames], lineage)
    
    # Position info for naming convention
    x_pos = batch_no + 1
    y_pos = 1
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(X.shape[0]):
        name_raw = os.path.join(raw_subdir, name_template.format('nuclear', x_pos, y_pos, i+1))
        name_gt_mask = os.path.join(seg_gt_subdir, name_template.format('mask-gt', x_pos, y_pos, i+1))
        name_dc_mask = os.path.join(seg_dc_subdir, name_template.format('mask-dc', x_pos, y_pos, i+1))
        
        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_gt_mask, y[i, ..., channel])
        imwrite(name_dc_mask, y_pred[i, ..., channel])

2022-12-01 00:24:15.382810: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima

# 2. Tracking
1. Open Matlab > 2020 and open the TracX folder
2. From the Matlab terminal, run the following 
`addpath(genpath('tracx'));`
3. Open the `run_tracx.m` script in matlab and press run.

# 3. Evaluate

In [14]:
import glob
import os
import subprocess

import numpy as np
import pandas as pd
from tifffile import imread, imwrite

from deepcell_tracking.isbi_utils import load_tiffs
from deepcell_tracking.metrics import TrackingMetrics

In [11]:
data_dir = 'data'
ctc_dir = os.path.join(data_dir, 'CTC')
ctc_dc_dir = os.path.join(ctc_dir, 'seg-dc')
ctc_gt_dir = os.path.join(ctc_dir, 'seg-gt')
seg_gt_dir = os.path.join(data_dir, 'seg-gt')
seg_dc_dir = os.path.join(data_dir, 'seg-dc')

ids = os.listdir(seg_gt_dir)

node_match_threshold = 0.6

ctc_software = '/publication-tracking/benchmarking/CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [9]:
def create_new_lineage(y):
    """Create a blank lineage dict for ids that have already been
    linked via IOU. Link only based on overlap,
    so there are no divisions/daughters/parents/deaths
    Args:
        y: (np.array) label image stack.
    Returns:
        dict: a nested dict (lineage for .trk)
    """
    new_lineage = {}
    for i, frame in enumerate(y):
        # Add to frames field if ID exists
        cells_in_frame = np.unique(frame)
        cells_in_frame = np.delete(
            cells_in_frame, np.where(cells_in_frame == 0)
        )
        cells_in_frame = list(cells_in_frame)

        for cell in cells_in_frame:
            cell = int(cell)
            if cell in new_lineage:
                new_lineage[cell]["frames"].append(i)

            # Or create a new dict because its a new cell
            else:
                new_lineage[cell] = {
                    "label": cell,
                    "frames": [i],
                    "daughters": [],
                    "capped": False,
                    "frame_div": None,
                    "parent": None,
                }

    return new_lineage

Load each movie, relabel the mask to match the new tracking ids and create the lineage information

In [16]:
for res_dir in [seg_gt_dir, seg_dc_dir]:
    for i in ids:    
        print(res_dir, i)
        # Find results path
        cc_path = glob.glob(os.path.join(res_dir, i, '*CellCycleResults*'))[0]
        t_path = glob.glob(os.path.join(res_dir, i, '*TrackingResults*'))[0]

        # Load TracX results
        cc_res = pd.read_csv(cc_path, sep='\t')
        t_res = pd.read_csv(t_path, sep='\t')

        y_old = load_tiffs(os.path.join(res_dir, i))

        # Create a new y array with update cell ids
        y_new = np.zeros_like(y_old)
        for _, r in t_res.iterrows():
            id_new = int(r['track_index'])
            id_old = int(r['cell_index'])
            t = int(r['cell_frame']) - 1 # Time indexed starting at 1 in TracX
            y_new[t][y_old[t] == id_old] = id_new

        # Create lineage
        lineage = create_new_lineage(y_new)

        # Assign parents and daughters
        for _, r in cc_res.iterrows():
            tid = r['track']
            # Check if a parent should be added
            if r['parent'] != 0:
                lineage[tid]['parent'] = int(r['parent'])

            # Check of daughter should be added
            if not np.isnan(r['daughter']):
                lineage[tid]['daughters'].append(int(r['daughter']))
                lineage[tid]['frame_div'] = max(lineage[tid]['frames'])
                
        # Correct discontiguous tracks, which are not allowed by CTC
        y_new, lineage = utils.convert_to_contiguous(y_new, lineage)
            
        # Save results in CTC format
        utils.save_ctc_res(os.path.join(ctc_dir, os.path.basename(res_dir)), int(i), y_new, lineage)

data/seg-gt 011
data/seg-gt 002
data/seg-gt 009
data/seg-gt 003
data/seg-gt 012
data/seg-gt 007
data/seg-gt 001
data/seg-gt 006
data/seg-gt 010
data/seg-gt 004
data/seg-gt 008
data/seg-gt 005
data/seg-dc 011
data/seg-dc 002
data/seg-dc 009
data/seg-dc 003
data/seg-dc 012
data/seg-dc 007
data/seg-dc 001
data/seg-dc 006
data/seg-dc 010
data/seg-dc 004
data/seg-dc 008
data/seg-dc 005


In [17]:
benchmarks = []

for results_dir, s in zip([ctc_gt_dir, ctc_dc_dir], ['GT', 'Deepcell']):
    for data_id in ids:
        print(data_id)
        results = {
            'model': f'TracX - {s}',
            'data_id': data_id
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)
        
        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout
            
            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))
        
        benchmarks.append(results)
        
df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

011
missed node 14_10 division completely
missed node 17_24 division completely
missed node 18_9 division completely
21_42 out degree = 1, daughters mismatch.
25_36 out degree = 2, parents mismatch, gt and res degree equal.
38_5 out degree = 1, daughters mismatch.
missed node 41_26 division completely
missed node 43_4 division completely
46_35 out degree = 1, daughters mismatch.
missed node 49_6 division completely
missed node 50_42 division completely
54_28 out degree = 1, daughters mismatch.
missed node 57_12 division completely
corrected division 14_10 as a frameshift division not an error
002
missed node 21_19 division completely
009
missed node 5_6 division completely
missed node 10_66 division completely
missed node 15_66 division completely
16_9 out degree = 1, daughters mismatch.
missed node 22_56 division completely
missed node 25_57 division completely
28_39 out degree = 1, daughters mismatch.
missed node 30_37 division completely
missed node 35_52 division completely
missed 

In [18]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,TracX - GT,011,3,5,4,7,15,3660,3833,3802,3975,1.000000,1.000000,0.997925
1,TracX - GT,002,0,0,0,1,1,1075,1075,1109,1109,1.000000,1.000000,0.999764
2,TracX - GT,009,12,18,23,27,57,14555,14806,14923,15174,1.000000,1.000000,0.998969
3,TracX - GT,003,1,2,0,5,8,2017,2055,2084,2122,1.000000,1.000000,0.998726
4,TracX - GT,012,2,7,4,7,16,9544,9796,9845,10097,1.000000,1.000000,0.998444
5,TracX - GT,007,0,0,0,1,1,196,199,203,206,1.000000,1.000000,0.997036
6,TracX - GT,001,0,1,0,2,3,992,997,1025,1030,1.000000,1.000000,0.999407
7,TracX - GT,006,0,0,2,0,0,337,383,351,397,1.000000,1.000000,0.998020
8,TracX - GT,010,21,18,3,15,54,8904,9090,9171,9357,1.000000,1.000000,0.998500
9,TracX - GT,004,0,0,0,1,1,521,521,540,540,1.000000,1.000000,0.999596
